![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 3-MODUL. MA’LUMOTLARGA ISHLOV BERISH

### 3.1 Fayllar bilan ishlash

#### Ma'lumotlar ombori. 2-qism

# Mavjud DF ni Omborga yangi jadval ko'rinishida yuklash

Kerakli modullarni chaqiramiz

In [69]:
import pandas as pd
import sqlite3 as sql

Yangi ma'lumotlar omborini yaratamiz

In [70]:
db =sql.connect("newdb.db")
cursor = db.cursor()

Ishlash uchun onlayndan jadval yuklab olamiz

In [71]:
df = pd.read_csv("https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/Electricity_Production_By_Source.csv")
df.head()

,Entity,Code,Year,Electricity from coal (TWh),Electricity from gas (TWh),Electricity from hydro (TWh),Electricity from other renewables (TWh),Electricity from solar (TWh),Electricity from oil (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh)
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


DFni omborga yuklashdan avval, kelin ustunlar nomini qisqartiraylik. Bunday qilishimizga sabab, keyinchalik SQL komandalar yozganda uzun nomlarni yozish o'ziga yarasha noqulayliklar keltirib chiqaradi.

In [72]:
cols = ['country','code','year','coal','gas','hydro','renewable_other','solar','oil','wind','nuclear']
df.columns = cols
df.head()

,country,code,year,coal,gas,hydro,renewable_other,solar,oil,wind,nuclear
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


`.to_sql()` metodi yordamida jadvalni to'g'ridan to'g'ri omborga ko'chirishimiz mumkin.

Jadvalga `jadval` deb yangi nom beramiz.

In [78]:
df.to_sql('jadval',db)

ValueError: Table 'jadval' already exists.

Jadvalni qaytib o'qiymiz

In [ ]:
query = "SELECT * FROM jadval"
df2 = pd.read_sql(query,db, index_col="index")

In [ ]:
df2.head()

In [ ]:
df2.info()

# Omborda bo'sh jadval yaratish

**Jadval yaratish qadamlari:**
- `CREATE TABLE` - jadval yaratish buyrug'i
- `IF NOT EXISTS` - jadval mavjud bo'lmasa
- `worldpop` - jadval nomi
- Qavs ichida jadval ustunlari, ustundagi ma'lumot turi va qo'shimcha parametrlar vergul bilan ajratib yoziladi.

**Quyidagi misolada jadvalimiz 5 ta ustundan iborat:**
- `rank` - davlat reytingi (zichlik bo'yicha), `INT` ya'ni butun son.
- `country` - davlat nomi, `TEXT` - matn, `PRIMARY KEY` - birlamchi qiymat, qatorga shu ustun qiymati bilan murojaat qilinadi (takrorlanman bo'lishi kerak)
- `density` - zichlik, `DOUBLE` - o'nlik son
- `population` - aholi soni, `DOUBLE` - o'nlik son, `NOT NULL` - bo'sh bo'lmasligi (qiymat berilishi shart)
- `area` - maydoni, `INT` - butun son, `NOT NULL` - qiymat berilishi shart.

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS worldpop(
rank INT,
country TEXT PRIMARY KEY,
density DOUBLE,
population DOUBLE NOT NULL,
area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

Ahamiyat bering, endi o'zgartirish kiritgandan so'ng natijalarni xotiradan faylga yozish uchun `.commit()` metodiga murojaat qilayapmiz.

Jadval qo'shilganini tekshirib ko'ramiz

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

Jadval ustunlarini tekshirib ko'ramiz

In [ ]:
cursor.execute("PRAGMA table_info(worldpop);")
cursor.fetchall()

Jadvalni o'qiymiz

In [ ]:
query = "SELECT * FROM worldpop"
df3 = pd.read_sql(query,db)
df3

Hozircha jadval bo'sh.

## Jadvalga yangi ma'lumot qo'shish
SQL jadvlaga ushbu manzilda joylashgan ma'lumotlarni kiritamiz: https://github.com/anvarnarz/praktikum_datasets/blob/main/world_population.csv

![Imgur](https://i.imgur.com/lilTCTm.png)

Keling SQL jadvalga yuqoridan `Macau` davlatiga oid ma'lumotlarni kiritamiz.

Agar barcha ustunlar uchun qiymat berayotgan bo'lsak to'g'ridan-to'g'ri qiymatlarni yozamiz. Bunda qiymatlar tartibi jadval ustunlari ketma-ketligi bilan mos tushishiga ahamiyat bering:

In [ ]:
query = "INSERT INTO worldpop VALUES (1, 'Macau', 21946.4667, 658.3940, 30)"
cursor.execute(query)
db.commit()

Qiymat qo'shilganini tekshirib ko'ramiz:

In [ ]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

In [ ]:
pd.read_sql(query,db)

Agar faqatgina ma'lum ustunlarga qiymat qo'shilsa, avval ustun nomlari yoziladi. Keling jadvlaga O'zbekiston haqida quyidagi ma'lumotlarni qo'shamiz:
- `country` = `Uzbekistan`
- `population` = `33935.7630`
- `area` = `447400`

In [ ]:
query = """
INSERT INTO worldpop (country,population,area)
VALUES ('Uzbekistan', 33935.7630, 447400)
"""
cursor.execute(query)
db.commit()

Tekshirib ko'ramiz

In [ ]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

Ko'rib turganingizdek mavjud bo'lmagan qiymatlar o'rniga `None` qo'yb ketildi.

## Ma'lumotlarni yangilash/qo'shish
`Uzbekistan` qatoriga `rank` va `density` qiymatlarini qo'shamiz.

In [ ]:
query = """
UPDATE worldpop
SET rank=133, density=75.8511
WHERE country="Uzbekistan"
"""
cursor.execute(query)
db.commit()

Tekshirib ko'ramiz

In [ ]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

## Jadvaldan ma'lumot o'chirish

In [ ]:
query = 'DELETE FROM worldpop WHERE country="Macau"'
cursor.execute(query)
db.commit()

Tekshirib ko'ramiz

In [ ]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

## Jadvalni butunlay o'chirib tashlash

In [ ]:
cursor.execute('DROP TABLE worldpop')
db.commit()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# SQL buyruqlarni f-string yordamida yozish
Biz yuqorida SQL buyruqlarni oddiy matn ko'rinishida yozayapamiz, demak Pythondagi f-string yordamida matnlarga o'zgaruvchilarning qiymatini ham qo'shish mumkin.

Keling boshlanishiga quyidagi DF ni yaratib olamiz, so'ngra DF qatorlarini yangi SQL jadvalga f-string yordamida yozamiz.

In [79]:
df = pd.read_csv("https://github.com/anvarnarz/praktikum_datasets/raw/main/world_population.csv", index_col='rank')
df.head()

,country,density,densityMi,pop2021,area
rank,,,,,
1,Macau,21946.4667,56841.3487,658.394,30
2,Monaco,19755.5000,51166.7450,39.511,2
3,Singapore,8305.1915,21510.4461,5896.686,710
4,Hong Kong,6841.3134,17719.0017,7552.810,1104
5,Gibraltar,5616.3333,14546.3033,33.698,6


Yangi SQL jadval yaratamiz

In [80]:
query = """
CREATE TABLE IF NOT EXISTS worldpop(
rank INT,
country TEXT PRIMARY KEY,
density DOUBLE,
population DOUBLE NOT NULL,
area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

Keling jadvladan O'zbekiston uchun qiymatlarni ajratib olamiz

In [81]:
uz=df[df.country=="Uzbekistan"]
print(uz)

         country  density  densityMi    pop2021    area
rank                                                   
133   Uzbekistan  75.8511   196.4542  33935.763  447400


In [82]:
rank = uz.index.values[0]
country = uz.at[rank, 'country']
density = uz.at[rank, 'density']
population = uz.at[rank, 'pop2021']
area = uz.at[rank, 'area']
print(f"{rank=}, {country=}, {density=}, {population=}, {area=}")

rank=np.int64(133), country='Uzbekistan', density=np.float64(75.8511), population=np.float64(33935.763), area=np.int64(447400)


In [ ]:
query = f"""
INSERT INTO worldpop VALUES ({rank}, '{country}', {density}, {population}, {area})
"""
cursor.execute(query)
db.commit()

In [84]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

## Omborni yopamiz.
Ishni tugatib omborni yopishni unutmang.

In [87]:
db.close()